##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Transfer learning with TensorFlow Hub

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ja/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

[TensorFlow Hub](http://tensorflow.org/hub) は、訓練済みモデルコンポーネントを共有する手法のひとつです。訓練済みモデルを検索する場合は、こちらの [TensorFlow Module Hub](https://tfhub.dev/) をご覧ください。このチュートリアルでは、次の内容のデモを行います:

1. TensorFlow Hub を `tf.keras` でどのように利用するか。
1. TensorFlow Hub を用いてどのように画像分類を行うか。
1. シンプルな転移学習をどのように行うか。

## セットアップ

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt

try:
  # %tensorflow_version は Colab にのみ存在します。
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf

In [0]:
!pip install -U tf-hub-nightly
import tensorflow_hub as hub

from tensorflow.keras import layers

## ImageNet の分類器

### 分類器のダウンロード

mobilenet をロードするために `hub.module` を、そして keras のレイヤーとして仕上げるために `tf.keras.layers.Lambda` を使用します。tfhub.dev にある、いずれの [TensorFlow 2 互換の画像分類器の URL](https://tfhub.dev/s?q=tf2&module-type=image-classification) でも、こちらで動作します。

In [0]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}

In [0]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

### 1 枚の画像で実行する

モデルを試すために、画像を 1 枚ダウンロードします。

In [0]:
import numpy as np
import PIL.Image as Image

grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

In [0]:
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

バッチの次元を追加し、画像をモデルに入力します。

In [0]:
result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

この結果はロジットの 1001 要素の配列で、画像がそれぞれのクラスである確率に基づく数値が割り振られ、順位付けされています。

そのため、トップのクラス ID は argmax を使うことでみつけることができます:

In [0]:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

### 推論結果のデコード

ここまでの内容で推論結果のクラス ID を表す数値が得られるので、
`ImageNet` のラベルを取得し、推論結果をデコードします

In [0]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [0]:
plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## シンプルな転移学習

TF Hub を利用することで、データセットのクラスを認識するための、モデルの最上位層を簡単に再訓練することができます。

### データセット

こちらの例では、TensorFlow flowers データセットを利用して進めます:

In [0]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

このデータをモデルにロードするには `tf.keras.preprocessing.image.ImageDataGenerator` を使うのがもっとも簡単な方法です。

すべての TensorFlow Hub の画像モジュールは `[0, 1]` の範囲で float で入力されることを想定しています。入力をリスケールする際には `ImageDataGenerator` の `rescale` パラメータを利用してください。

画像のサイズは後ほど処理されます。

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(str(data_root), target_size=IMAGE_SHAPE)

結果のオブジェクトは `image_batch, label_batch` のペアを返すイテレーターです。

In [0]:
for image_batch, label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

### 分類器で画像をバッチ処理する

分類器で画像をバッチ処理していきます。

In [0]:
result_batch = classifier.predict(image_batch)
result_batch.shape

In [0]:
predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

次に、これらの推論結果が画像とどのように一致するかを確認します:

In [0]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

画像の帰属については `LICENSE.txt` をご参照ください。

完璧とは程遠い結果ですが、このモデルはこれらのクラスのために訓練されたものではない（ただし、"daisy" を除いて）ということを考慮すると、妥当な結果だといえるでしょう。

### ヘッドレスモデルのダウンロード

TensorFlow Hub は最上位の分類層を含まないモデルも配布しています。これらは転移学習に簡単に利用することができます。

tfhub.dev にある、いずれの [TensorFlow 2 互換の image feature vector URL](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2) でも、こちらで動作します。

In [0]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}

特徴抽出器 (feature extractor) を作成します。

In [0]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224,224,3))

これは画像毎に長さ 1280 のベクトルデータを返します:

In [0]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

特徴抽出レイヤーの変数をフリーズして、訓練が新しい分類器のレイヤーのみを変更するようにします。

In [0]:
feature_extractor_layer.trainable = False

### 上位の分類層を接合する

`tf.keras.Sequential` モデルの hub レイヤーをラップして、新しい分類層を追加します。

In [0]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(image_data.num_classes, activation='softmax')
])

model.summary()

In [0]:
predictions = model(image_batch)

In [0]:
predictions.shape

### モデルの訓練

訓練プロセスのコンフィグにはコンパイルを使用します:

In [0]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'])

モデルの訓練には `.fit` メソッドを使用します。

この例を短く保つために訓練は 2 エポックだけにします。訓練のプロセスを可視化するために、各エポックの平均だけではなく各々のバッチで個別に損失と正確度を記録するためのカスタムコールバックを使用します。

In [0]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [0]:
steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit_generator(image_data, epochs=2,
                              steps_per_epoch=steps_per_epoch,
                              callbacks = [batch_stats_callback])

これで、ほんの数回の訓練の繰り返しでさえ、タスクにおいてモデルが進歩していることが分かりました。

In [0]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

In [0]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

### 推論結果の確認

前からプロットをやり直すには、まずクラス名のリストを取得します:

In [0]:
class_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

画像のバッチをモデルに入力し、得られた ID をクラス名に変換します。

In [0]:
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

結果をプロットします

In [0]:
label_id = np.argmax(label_batch, axis=-1)

In [0]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

## モデルのエクスポート

訓練が完了したので、saved model としてエクスポートします:

In [0]:
import time
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path, save_format='tf')

export_path

リロードできること、そしてエクスポートする前とおなじ結果が得られることを確認します:

In [0]:
reloaded = tf.keras.models.load_model(export_path)

In [0]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [0]:
abs(reloaded_result_batch - result_batch).max()

この saved model は後々推論を行うため、そして [TFLite](https://www.tensorflow.org/lite/convert/) や [TFjs](https://github.com/tensorflow/tfjs-converter) のモデルに変換するためにロードできます。

